In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from math import ceil
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# Model configuration
img_width, img_height = 28, 28
batch_size = 250
no_epochs = 20
no_classes = 10
validation_split = 0.2
verbosity = 1
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()

# Reshape data
input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)
    
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

Using TensorFlow backend.


## Width and Height Shift

In [2]:
# specify the width and height shift arguments
width_shift_val = 0.0
height_shift_val = 0.0

# create the class object
datagen = ImageDataGenerator(width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 4s 18ms/step - loss: 0.3210 - acc: 0.9039
Epoch 2/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0845 - acc: 0.9741
Epoch 3/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0609 - acc: 0.9817
Epoch 4/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0465 - acc: 0.9849
Epoch 5/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0421 - acc: 0.9865
Epoch 6/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0336 - acc: 0.9898
Epoch 7/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0316 - acc: 0.9899
Epoch 8/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0277 - acc: 0.9914
Epoch 9/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0244 - acc: 0.9921
Epoch 10/20
240/240 [==============================] - 2s 7ms/step - loss: 0.0210 - acc: 0.9929
Epoch 11/20
240/240 [===========================

## Rotation

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 60

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 9s 36ms/step - loss: 0.5439 - acc: 0.8273
Epoch 2/20
240/240 [==============================] - 7s 27ms/step - loss: 0.1887 - acc: 0.9407
Epoch 3/20
240/240 [==============================] - 7s 27ms/step - loss: 0.1373 - acc: 0.9564
Epoch 4/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1139 - acc: 0.9651
Epoch 5/20
240/240 [==============================] - 7s 29ms/step - loss: 0.0965 - acc: 0.9696
Epoch 6/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0873 - acc: 0.9725
Epoch 7/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0796 - acc: 0.9751
Epoch 8/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0722 - acc: 0.9777
Epoch 9/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0695 - acc: 0.9789
Epoch 10/20
240/240 [==============================] - 7s 27ms/step - loss: 0.0639 - acc: 0.9798
Epoch 11/20
240/240 [==================

## Shear

In [2]:
# specify the shear argument
shear_range_val=45

# create the class object
datagen = ImageDataGenerator(shear_range=shear_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 9s 36ms/step - loss: 0.4631 - acc: 0.8547
Epoch 2/20
240/240 [==============================] - 6s 26ms/step - loss: 0.1530 - acc: 0.9532
Epoch 3/20
240/240 [==============================] - 6s 26ms/step - loss: 0.1123 - acc: 0.9639
Epoch 4/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0907 - acc: 0.9711
Epoch 5/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0781 - acc: 0.9752
Epoch 6/20
240/240 [==============================] - 6s 27ms/step - loss: 0.0686 - acc: 0.9786
Epoch 7/20
240/240 [==============================] - 6s 27ms/step - loss: 0.0594 - acc: 0.9815
Epoch 8/20
240/240 [==============================] - 7s 27ms/step - loss: 0.0574 - acc: 0.9814
Epoch 9/20
240/240 [==============================] - 6s 26ms/step - loss: 0.0524 - acc: 0.9834
Epoch 10/20
240/240 [==============================] - 6s 26ms/step - loss: 0.0489 - acc: 0.9847
Epoch 11/20
240/240 [==================

## Zoom

In [2]:
# specify the zoom argument
zoom_range_val=[0.5,1.5]

# create the class object
datagen = ImageDataGenerator(zoom_range=zoom_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 8s 35ms/step - loss: 0.5073 - acc: 0.8426
Epoch 2/20
240/240 [==============================] - 6s 26ms/step - loss: 0.1711 - acc: 0.9480
Epoch 3/20
240/240 [==============================] - 6s 25ms/step - loss: 0.1284 - acc: 0.9603
Epoch 4/20
240/240 [==============================] - 7s 28ms/step - loss: 0.1049 - acc: 0.9671
Epoch 5/20
240/240 [==============================] - 7s 28ms/step - loss: 0.0910 - acc: 0.9720
Epoch 6/20
240/240 [==============================] - 7s 29ms/step - loss: 0.0820 - acc: 0.9740
Epoch 7/20
240/240 [==============================] - 6s 27ms/step - loss: 0.0765 - acc: 0.9766
Epoch 8/20
240/240 [==============================] - 6s 27ms/step - loss: 0.0683 - acc: 0.9779
Epoch 9/20
240/240 [==============================] - 6s 26ms/step - loss: 0.0677 - acc: 0.9789
Epoch 10/20
240/240 [==============================] - 6s 27ms/step - loss: 0.0615 - acc: 0.9809
Epoch 11/20
240/240 [==================

## Rotate + W&H Shift

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 30

# specify the width and height shift arguments
width_shift_val = 0.1
height_shift_val = 0.1

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 11s 44ms/step - loss: 0.6743 - acc: 0.7837
Epoch 2/20
240/240 [==============================] - 8s 33ms/step - loss: 0.2371 - acc: 0.9267
Epoch 3/20
240/240 [==============================] - 8s 35ms/step - loss: 0.1685 - acc: 0.9476
Epoch 4/20
240/240 [==============================] - 8s 32ms/step - loss: 0.1354 - acc: 0.9580
Epoch 5/20
240/240 [==============================] - 8s 34ms/step - loss: 0.1161 - acc: 0.9638
Epoch 6/20
240/240 [==============================] - 8s 34ms/step - loss: 0.1041 - acc: 0.9672
Epoch 7/20
240/240 [==============================] - 8s 34ms/step - loss: 0.0946 - acc: 0.9705
Epoch 8/20
240/240 [==============================] - 8s 34ms/step - loss: 0.0865 - acc: 0.9725
Epoch 9/20
240/240 [==============================] - 8s 33ms/step - loss: 0.0804 - acc: 0.9740
Epoch 10/20
240/240 [==============================] - 8s 33ms/step - loss: 0.0758 - acc: 0.9761
Epoch 11/20
240/240 [=================

## Rotate + W&H Shift + Shear

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 30

# specify the shear argument
shear_range_val = 25

# specify the width and height shift arguments
width_shift_val = 0.1
height_shift_val = 0.1

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 11s 44ms/step - loss: 0.7904 - acc: 0.7416
Epoch 2/20
240/240 [==============================] - 8s 33ms/step - loss: 0.2973 - acc: 0.9072
Epoch 3/20
240/240 [==============================] - 8s 33ms/step - loss: 0.2012 - acc: 0.9372
Epoch 4/20
240/240 [==============================] - 8s 34ms/step - loss: 0.1623 - acc: 0.9504
Epoch 5/20
240/240 [==============================] - 8s 35ms/step - loss: 0.1361 - acc: 0.9579
Epoch 6/20
240/240 [==============================] - 8s 35ms/step - loss: 0.1227 - acc: 0.9617
Epoch 7/20
240/240 [==============================] - 8s 35ms/step - loss: 0.1094 - acc: 0.9658
Epoch 8/20
240/240 [==============================] - 9s 36ms/step - loss: 0.1029 - acc: 0.9673
Epoch 9/20
240/240 [==============================] - 8s 35ms/step - loss: 0.0951 - acc: 0.9695
Epoch 10/20
240/240 [==============================] - 9s 36ms/step - loss: 0.0919 - acc: 0.9717
Epoch 11/20
240/240 [=================

## Rotate + W&H Shift + Shear + Zoom

In [2]:
# specify the maximum rotation_range angle
rotation_range_val = 30

# specify the shear argument
shear_range_val = 25

# specify the width and height shift arguments
width_shift_val = 0.1
height_shift_val = 0.1

# specify the zoom argument
zoom_range_val=[2.5,3.5]

# create the class object
datagen = ImageDataGenerator(rotation_range=rotation_range_val, width_shift_range=width_shift_val, height_shift_range=height_shift_val, shear_range=shear_range_val, zoom_range=zoom_range_val)

# fit the generator
datagen.fit(input_train.reshape(input_train.shape[0], 28, 28, 1))
# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(datagen.flow(input_train, target_train, batch_size=batch_size),
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/20
240/240 [==============================] - 12s 48ms/step - loss: 1.2361 - acc: 0.5862
Epoch 2/20
240/240 [==============================] - 9s 38ms/step - loss: 0.6992 - acc: 0.7712
Epoch 3/20
240/240 [==============================] - 10s 41ms/step - loss: 0.5854 - acc: 0.8105
Epoch 4/20
240/240 [==============================] - 9s 38ms/step - loss: 0.5183 - acc: 0.8307
Epoch 5/20
240/240 [==============================] - 10s 40ms/step - loss: 0.4779 - acc: 0.8444
Epoch 6/20
240/240 [==============================] - 10s 44ms/step - loss: 0.4443 - acc: 0.8555
Epoch 7/20
240/240 [==============================] - 9s 40ms/step - loss: 0.4179 - acc: 0.8643
Epoch 8/20
240/240 [==============================] - 9s 39ms/step - loss: 0.4031 - acc: 0.8679
Epoch 9/20
240/240 [==============================] - 9s 39ms/step - loss: 0.3836 - acc: 0.8762
Epoch 10/20
240/240 [==============================] - 9s 39ms/step - loss: 0.3632 - acc: 0.8821
Epoch 11/20
240/240 [==============